## Example SpikeForest import

This notebook will import a few SpikeForest recordings into the labbox_ephys database

In [ ]:
# Define some functions

import lbdb
import kachery as ka
from spikeforest2_utils import MdaRecordingExtractor, MdaSortingExtractor, AutoRecordingExtractor, AutoSortingExtractor
import hither
import numpy as np

def geom_from_recording(recording):
    channel_ids = recording.get_channel_ids()
    location0 = recording.get_channel_property(channel_ids[0], 'location')
    nd = len(location0)
    M = len(channel_ids)
    geom = np.zeros((M, nd))
    for ii in range(len(channel_ids)):
        location_ii = recording.get_channel_property(channel_ids[ii], 'location')
        geom[ii, :] = list(location_ii)
    return geom

def import_recording(*, recording_id, recording_path, sorting_true_path=None):
    print(f'Importing {recording_id}')
    recording = AutoRecordingExtractor(recording_path, download=True)
    doc = dict(
        recording_id=recording_id,
        recording_path=recording_path,
        sampling_frequency=recording.get_sampling_frequency(),
        channel_ids=recording.get_channel_ids(),
        geom=geom_from_recording(recording).tolist(),
        num_frames=recording.get_num_frames()
    )
    lbdb.update(collection='recordings', query=dict(recording_id=recording_id), update=doc, upsert=True)
    if sorting_true_path is not None:
        sorting_true = AutoSortingExtractor(sorting_true_path)
        sorting_true_doc=dict(
            sorting_path=sorting_true_path,
            unit_ids=sorting_true.get_unit_ids().tolist()
        )
        lbdb.update(collection='recordings', query=dict(recording_id=recording_id), update={'$set': dict(sorting_true=sorting_true_doc)})

In [ ]:
# Auto import local recordings

def auto_import_recordings(dirpath, prefix):
    print(f'Checking {dirpath}')
    x = ka.read_dir(dirpath)
    for dirname, dd in x['dirs'].items():
        if 'raw.mda' in dd['files'] and 'params.json' in dd['files'] and 'geom.csv' in dd['files']:
            recording_path = ka.store_dir(dirpath + '/' + dirname)
            import_recording(recording_id=f'{prefix}{dirname}', recording_path=recording_path)
        else:
            auto_import_recordings(dirpath + '/' + dirname, prefix=prefix + dirname + '/')

auto_import_recordings(dirpath='/data/local-data', prefix='local:')

In [ ]:
# Now list all of the recordings in the database

recordings = lbdb.find(collection='recordings', query=dict())
for rec in recordings:
    print(rec['recording_id'])

In [ ]:
# Write a toy example recording for testing
# import spikeextractors as se
# from spikeforest2_utils import MdaRecordingExtractor
# R, S = se.example_datasets.toy_example()
# MdaRecordingExtractor.write_recording(recording=R, save_path='toy_example1')